IMPORTING LIBRARIES

In [1]:
import os
import pandas as pd
import numpy as np
import librosa
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences

LOADING THE DATASET

In [2]:
dataset_path="E:/final year project/dataset/tess"
dataset_map="E:/final year project/dataset/tess/speech_emotion_recognisition.csv"
if not os.path.exists(dataset_map):
    print("Data csv file couldnt be loaded")
elif not os.path.exists(dataset_path):
    print("Data couldn't be loaded")
else:
    print("Data loaded succesfully")

Data loaded succesfully


In [3]:
speech_df=pd.read_csv(dataset_map)
# speech_df=speech_df[speech_df['Emotion']!='Surprised']
# speech_df=speech_df[speech_df['Emotion']!='Calm']
speech_df = speech_df.drop(speech_df.columns[0], axis=1)
print(speech_df)

      Emotion                     Filename
0       Angry           OAF_back_angry.wav
1       Angry            OAF_bar_angry.wav
2       Angry           OAF_base_angry.wav
3       Angry           OAF_bath_angry.wav
4       Angry           OAF_bean_angry.wav
...       ...                          ...
16795     Sad  YAF_witch_sad_augmented.wav
16796     Sad  YAF_yearn_sad_augmented.wav
16797     Sad    YAF_yes_sad_augmented.wav
16798     Sad  YAF_young_sad_augmented.wav
16799     Sad  YAF_youth_sad_augmented.wav

[16800 rows x 2 columns]


FEATURE EXTRACTION

DEFINING FUNCTION FO ZCR

In [4]:
def get_zcr(y,frame_length, hop_length,maxlen):
    zcr=librosa.feature.zero_crossing_rate(y=y, frame_length=frame_length, hop_length=hop_length)
    return pad_sequences([zcr.T], maxlen=maxlen,truncating="post",padding="post")
# y,sr =librosa.load("E:/final year project/dataset/Calm/03-01-02-01-01-01-01.wav")
# print(y)
# zcr=get_zcr(y=y,frame_length=2048,hop_length=512,maxlen=130)
# print(zcr.shape)

DEFINING FUNCTION FOR RMS ENERGY

In [5]:
def get_rms(y,frame_length, hop_length,maxlen):
    rms=librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)
    return pad_sequences([rms.T], maxlen=maxlen,truncating="post",padding="post")

# rms=get_rms(y=y,frame_length=2048,hop_length=512,maxlen=130)
# print(rms.shape)

DEFINING FUNCTION FOR MFCC

In [6]:
# def get_mfcc(y,sr,n_mfcc,hop_length,maxlen):
def get_mfcc(y,sr,n_mfcc):

    # mfcc=librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc,hop_length=hop_length)
    mfcc=librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfcc
    # return pad_sequences([mfcc.T], maxlen=maxlen,truncating="post",padding="post")
# mfcc=get_mfcc(y=y,sr=16000,n_mfcc=13,hop_length=512,maxlen=130)
# print(mfcc.shape)


DEFINING MELSPECTOGRAM FUNCTION

In [7]:
# def get_melspec(y,sr,n_fft,n_mels,hop_length,fmin,maxlen):
def get_melspec(y,sr):
    # mel_spec=librosa.feature.melspectrogram(y=y,sr=sr,n_fft=n_fft,n_mels=n_mels,hop_length=hop_length,fmin=fmin)
    mel_spec=librosa.feature.melspectrogram(y=y,sr=sr)
    # mel_spec_db=librosa.power_to_db(mel_spec,ref=np.max)
    return mel_spec
    #return pad_sequences(mel_spec_db,maxlen=maxlen,truncating="post",padding="post")
# mel_spec_db=get_melspec(y,sr,1024,64,512,50,130)
# print(mel_spec_db.shape)

COLLECTING THE FEATURES

In [8]:
# Defining the params value
FRAME_LENGTH=2048
HOP_LENGTH=512
N_MFCC=40
N_FFT=1024
MAX_LENGTH=130
N_MELS=128
FMIN=50
zcr_list=[]
rmse_list=[]
mfcc_list=[]
mel_spec_list=[]
labels=[]
# Iterating over each row
for index, row in speech_df.iterrows():
    path=os.path.join(dataset_path,row['Emotion'],row['Filename'])
    y,sr=librosa.load(path)
    # zcr=get_zcr(y,FRAME_LENGTH,HOP_LENGTH,MAX_LENGTH)
    # zcr_list.append(zcr)
    # rmse=get_rms(y,FRAME_LENGTH,HOP_LENGTH,MAX_LENGTH)
    # rmse_list.append(rmse)
    mfcc=get_mfcc(y,sr,N_MFCC)
    mfcc_list.append(np.mean(mfcc.T,axis=0))
    mel_spec=get_melspec(y,sr)
    mel_spec_list.append(np.mean(mel_spec.T,axis=0))
    labels.append(row['Emotion'])
    print("Values calculated for ",index)

    


Values calculated for  0
Values calculated for  1
Values calculated for  2
Values calculated for  3
Values calculated for  4
Values calculated for  5
Values calculated for  6
Values calculated for  7
Values calculated for  8
Values calculated for  9
Values calculated for  10
Values calculated for  11
Values calculated for  12
Values calculated for  13
Values calculated for  14
Values calculated for  15
Values calculated for  16
Values calculated for  17
Values calculated for  18
Values calculated for  19
Values calculated for  20
Values calculated for  21
Values calculated for  22
Values calculated for  23
Values calculated for  24
Values calculated for  25
Values calculated for  26
Values calculated for  27
Values calculated for  28
Values calculated for  29
Values calculated for  30
Values calculated for  31
Values calculated for  32
Values calculated for  33
Values calculated for  34
Values calculated for  35
Values calculated for  36
Values calculated for  37
Values calculated for 

CONVERTING LIST INTO NUMPY ARRAY

In [9]:
zcr_array=np.array(zcr_list)
rmse_array=np.array(rmse_list)
mfcc_array=np.array(mfcc_list)
mel_spec_array=np.array(mel_spec_list)
print(mel_spec_array.shape)
print(mfcc_array.shape)

(16800, 128)
(16800, 40)


In [10]:
# mel_spec_array = mel_spec_array.reshape(8498, 1, 130, 128)  # (samples, channels, time, freq_bins)
# mfcc_array = mfcc_array.reshape(8498, 1, 130, 13)  # (samples, channels, time, coefficients)

features=np.concatenate([mfcc_array,mel_spec_array],axis=-1)
features=features.astype('float32')
Emotions=np.asarray(labels)

features.shape

(16800, 168)

In [11]:
# features=features.reshape(features.shape[0],-1)
# features_df=pd.DataFrame(features)
# features_df['Labels']=Emotions

In [12]:
# features_csv=features_df.to_csv(os.path.join(dataset_path,"Features.csv"))

In [13]:
# features_df.shape

In [14]:
# melspec_db_features=mel_spec_array.reshape(mel_spec_array.shape[0],-1)
# melspec_db_df=pd.DataFrame(melspec_db_features)
# mel_spec_array.shape

DATA PREPROCESSING

In [15]:
# def normalise_features(feature):
#     mean=np.mean(feature,axis=0)
#     std=np.std(feature,axis=0)
#     return (feature-mean)/(std+1e-9)
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
def normalisied_features(feature):
    scaled_features=scaler.fit_transform(feature)
    return scaled_features

In [16]:
normalisied_features=normalisied_features(features)

In [17]:

# print(normalisied_melspecs.shape)
# print(normalisied_melspecs)
normalisied_features=np.array(normalisied_features)
# print(normalisied_melspecs.shape)
# print(normalisied_melspecs)
print(normalisied_features.shape)


# print(np.mean(normalisied_features))
# print(np.std(normalisied_features))  


(16800, 168)


ENCODING THE EMOTION LABELS

In [18]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import joblib
label_encoder=LabelEncoder()
emotion_encoded=label_encoder.fit_transform(Emotions)
emotion_oneHot=to_categorical(emotion_encoded)
# print(emotion_encoded)

print(emotion_oneHot.shape)


(16800, 7)


In [20]:
artifacts_path='E:/final year project/model_artifacts'
os.makedirs(artifacts_path, exist_ok=True)
np.save(os.path.join(artifacts_path,'normalized_features.npy'), normalisied_features)
np.save(os.path.join(artifacts_path,'encoded_labels.npy'), emotion_oneHot)

joblib.dump(scaler, os.path.join(artifacts_path,'minmax_scaler.pkl'))
joblib.dump(label_encoder, os.path.join(artifacts_path,'label_encoder.pkl'))

['E:/final year project/model_artifacts\\label_encoder.pkl']

CREATING DATA FOR TRAIN TEST SPLIT

CREATING A MODEL

MODEL TRAINNING

ACCESSING ACCURACY

MODEL EVALUATION